In [1]:
import pandas as pd
import altair as alt
from datetime import date
import os

# PAST PERFORMANCE VS. PREDICTION

## Altair code

In [2]:
url = os.path.join(os.path.pardir,"logs","predicted_stats.json")
cutoff_path = os.path.join(os.path.pardir,"logs","season_cutoffs.pkl")
cutoffs = pd.read_pickle(cutoff_path)

In [3]:
url

'./logs/predicted_stats.json'

In [4]:
#player dropdown menu
eligible_path = os.path.join(os.path.pardir, "logs", "eligible_players.pkl")
players = pd.read_pickle(eligible_path)
players = players.PLAYER_NAME.sort_values().unique()
player_dropdown = alt.binding_select(
    options=players,
    name='Player: '
)
player_selection = alt.selection_single(
    fields=['PLAYER_NAME'],
    bind=player_dropdown,
    init={'PLAYER_NAME':'RJ Barrett'}
)

#radio filter
stat = ['PTS', 'REB', 'AST', 'FG3M', 'BLK', 'STL']
stat_group = [[s, s+"_PREDICT"] for s in stat]

stat_radio = alt.binding_radio(
    options=stat_group,
    name="Stat: ",
    labels=stat
)
stat_selection = alt.selection_single(
    fields=['MODEL'],
    bind=stat_radio
)

#get season domain to limit initial plotting
domain = [cutoffs.loc[cutoffs['PLAYOFFS']=='Regular Season']['start'].max(),
          date.today().strftime("%Y-%m-%d")]

#make chart of points (actual vs. predicted)
chart = (
    alt.Chart(url, title='Actual vs. Predicted Points')
    .mark_line(point=True)
    .encode(
        x = alt.X('GAME_DATE:T', title='Game date', timeUnit='yearmonthdate', scale=alt.Scale(domain=domain)),
        y = alt.Y('VALUE:Q', title='Points'),
        color = 'MODEL:N',
        tooltip=['PLAYER_NAME:N','MODEL:N','VALUE:Q', 'GAME_DATE:T', 'OPPONENT:N']
    )
    .add_selection(player_selection)
    .transform_filter(player_selection)
    .add_selection(stat_selection)
    .transform_filter(stat_selection)
    .interactive()
)

#make chart of playoff vs. regular season regions to overlay

playoffs = (
    alt.Chart(cutoffs)
    .mark_rect(opacity=0.15)
    .encode(
        x='start:T',
        x2 = 'stop:T',
        y=alt.value(0),
        y2=alt.value(300),
        color=alt.Color('PLAYOFFS:N', legend=alt.Legend(title="Legend"))
    )
)

final_chart = alt.layer(
    chart,
    playoffs
).properties(width=800, height=300)

## Past performance chart

In [6]:
final_chart

alt.LayerChart(...)

# BEST BET FOR UPCOMING GAME

## Altair code

In [7]:
best_bet_path = os.path.join(os.path.pardir, "logs", f"best_bet.pkl")
df = pd.read_pickle(best_bet_path)

#radio filter
stat = ['PTS', 'REB', 'AST', 'FG3M', 'BLK', 'STL']

stat_radio = alt.binding_radio(
    options=stat_group,
    name="Stat: ",
    labels=stat
)
stat_selection = alt.selection_single(
    fields=['STAT'],
    bind=stat_radio,
    init={'STAT':'PTS'}
)

#make bar chart
best_bet_chart = (
    alt.Chart(df, title="Best bet predictor")
    .mark_bar()
    .encode(
        x=alt.X("PLAYER_NAME:N", sort='y', title='Player'),
        y=alt.Y("dff:Q", title='Score'),
        color=alt.condition(
            alt.datum.dff > 0,
            alt.value("steelblue"),
            alt.value("orange")
        ),
        tooltip=["GAME_DATE:T","LINE:Q","PREDICT:Q"]
    )
    .add_selection(stat_selection)
    .transform_filter(stat_selection)
    .properties(width=400,height=300)
)

## BEST BET CHART

In [8]:
best_bet_chart

alt.Chart(...)